### Данный проект предназначен исключительно для образовательных целей. Мы не поддерживаем и не одобряем использование токсичных комментариев или языка, который может причинить вред другим. 
### Пожалуйста, используйте полученные результаты ответственно.

In [1]:
%pip install -q 'numpy<2' pandas scikit-learn torch transformers datasets

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd

import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification

In [8]:
# https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments
df = pd.read_csv('../../data/labeled.csv')

In [9]:
# generated with cursor.ai + gpt-4o-mini
toxic_sentences = [
    "Ты никогда не сможешь добиться успеха, потому что ты слишком глуп.",
    "Все вокруг тебя только и делают, что смеются над твоими неудачами.",
    "Ты не заслуживаешь любви и уважения, потому что ты никчемный человек.",
    "Твои идеи всегда проваливаются, и никто не хочет с тобой работать.",
    "Ты просто паразит, который живет за счет других людей.",
    "Ты всегда будешь неудачником, и это не изменится.",
    "Никто не хочет с тобой общаться, потому что ты слишком скучен.",
    "Ты не способен на что-то великое, просто смирись с этим.",
    "Твои попытки выглядеть умным только вызывают смех.",
    "Ты никому не нужен, и это печальная правда, чмо."
]

positive_sentences = [
    "Ты действительно талантливый человек, и у тебя есть все шансы на успех.",
    "Я восхищаюсь твоей работой и тем, как ты справляешься с трудностями.",
    "Ты всегда поддерживаешь других, и это делает тебя особенным.",
    "Твои идеи вдохновляют, и я уверен, что они принесут много пользы.",
    "Ты обладаешь уникальными способностями, которые могут изменить мир к лучшему."
]

In [10]:
toxicity_tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')
toxicity_model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')

/Users/y.korobko/Desktop/qwerty123/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
inappropriate_tokenizer = AutoTokenizer.from_pretrained("apanc/russian-inappropriate-messages")
inappropriate_model = AutoModelForSequenceClassification.from_pretrained("apanc/russian-inappropriate-messages")

In [18]:
s_tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
s_model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/y.korobko/Desktop/qwerty123/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def text2toxicity(text, aggregate=1):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = toxicity_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(toxicity_model.device)
        proba = torch.sigmoid(toxicity_model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

In [28]:
%%timeit
s_model(**s_tokenizer('Ужасный токсичный текст фу', return_tensors='pt'))

30 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
%%timeit
text2toxicity("Ужасный токсичный текст фу")

3 ms ± 380 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit
inappropriate_inputs = inappropriate_tokenizer("Ужасный токсичный текст фу", return_tensors="pt", padding=True, truncation=True)
inappropriate_score = inappropriate_model(**inappropriate_inputs)

35.1 ms ± 7.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
toxicity_model_params = sum(p.numel() for p in toxicity_model.parameters() if p.requires_grad)
inappropriate_model_params = sum(p.numel() for p in inappropriate_model.parameters() if p.requires_grad)
s_model_params = sum(p.numel() for p in s_model.parameters() if p.requires_grad)

print(f"Learnable parameters in toxicity_model: {toxicity_model_params / 1_000_000:.2f} million")
print(f"Learnable parameters in inappropriate_model: {inappropriate_model_params / 1_000_000:.2f} million")
print(f"Learnable parameters in s_model: {s_model_params / 1_000_000:.2f} million")


Learnable parameters in toxicity_model: 11.79 million
Learnable parameters in inappropriate_model: 177.85 million
Learnable parameters in s_model: 177.85 million


In [33]:
sampled_df = df.sample(n=20, random_state=21)
inputs = inappropriate_tokenizer(sampled_df['comment'].tolist(), padding=True, truncation=True, return_tensors="pt")
inappropriate_scores = inappropriate_model(**inputs).logits.softmax(dim=1)[:, 1]
toxicity_scores = [text2toxicity(sentence) for sentence in sampled_df['comment'].tolist()]
s_inputs = s_tokenizer(sampled_df['comment'].tolist(), padding=True, truncation=True, return_tensors="pt")
s_scores = s_model(**s_inputs).logits.softmax(dim=1)[:, 1]

sampled_df['predicted_inappropriate'] = inappropriate_scores.tolist()
sampled_df['predicted_toxicity'] = toxicity_scores
sampled_df['predicted_s_toxicity'] = s_scores.detach().cpu().numpy().tolist()


print("Sampled Comments and Their Predicted Toxicity Scores:")
for index, row in sampled_df.iterrows():
    print(f"Comment: {row['comment']}") 
    print(f"Predicted Inappropriate: {row['predicted_inappropriate']}") 
    print(f"Predicted Toxicity: {row['predicted_toxicity']}")
    print(f"Predicted S-toxicity: {row['predicted_s_toxicity']}")
    print(f"Actual Toxicity: {row['toxic']}\n\n")


Sampled Comments and Their Predicted Toxicity Scores:
Comment: Машина постоянно у меня, в воскресенье аренда не списывается, аренда списывается со счета в ящере, по нему катаю только безнал, детский и комфорт, катаю его только 2т в день, остальное время катаю только диспечерские службы, Таксовик и 068,тариф меня устраивает, ну да где-то часов 12 в день, 4 часа ящер чтобы 2т накатать и 8 часов диспечерские 5-6т, 1500 бензин остальное моё

Predicted Inappropriate: 0.03985416144132614
Predicted Toxicity: 0.049443751828052385
Predicted S-toxicity: 0.0020437585189938545
Actual Toxicity: 0.0


Comment: ой блять, как же вы заебали один еблан впишется в тему и прилично устроится и всё!1 - ОН, БЛЯТЬ, ВС СДЕЛАЛ САМ!1 И ВСЕГО ДОБИЛСЯ САМ!1 не по стечению обстоятельств, не потому что попёрло, а потому что ОН ВЕЛИКИЙ САМ!1 и тут же другие васи, которым чуть меньше в жизни подогнали и чуть хуже устроили, начинают ему подпевать - ВОТ СМОТРИТЕ! ОН САМ ВСЕГО ДОБИЛСЯ!1 ссука, какие же вы имбецилы... вы 

In [ ]:
def test_message(message, max_length=None, print_message=True):
    toxicity_score = text2toxicity(message)
    inappropriate_inputs = inappropriate_tokenizer(message, return_tensors="pt", padding=True, truncation=True)
    inappropriate_score = inappropriate_model(**inappropriate_inputs).logits.softmax(dim=1)[0, 1]
    s_inputs = s_tokenizer(message, padding=True, truncation=True, return_tensors="pt")
    s_score = s_model(**s_inputs).logits.softmax(dim=1)[0, 1]
    max_length = max_length or len(message)
    alignment_length = max_length - len(message)
    if print_message:
        print(f"{message} {' ' * alignment_length} | Inappropriate: {inappropriate_score.item():.4f} | Toxicity: {toxicity_score:.4f} | S-toxicity: {s_score.item():.4f}")
    else:
        print(f"Inappropriate: {inappropriate_score.item():.4f} | Toxicity: {toxicity_score:.4f} | S-toxicity: {s_score.item():.4f}")
    # return toxicity_score, inappropriate_score)

В примере снизу обе модели плохо работают на некоторых токсичных предложениях

In [35]:
print("Toxic Sentences:")
for sentence in toxic_sentences:
    test_message(sentence, max_length=80)

print("\nPositive Sentences:")
for sentence in positive_sentences:
    test_message(sentence, max_length=80)


Toxic Sentences:
Ты никогда не сможешь добиться успеха, потому что ты слишком глуп.                | Inappropriate: 0.4725 | Toxicity: 0.0662 | S-toxicity: 0.4042
Все вокруг тебя только и делают, что смеются над твоими неудачами.                | Inappropriate: 0.0398 | Toxicity: 0.2950 | S-toxicity: 0.7879
Ты не заслуживаешь любви и уважения, потому что ты никчемный человек.             | Inappropriate: 0.2449 | Toxicity: 0.0187 | S-toxicity: 0.7088
Твои идеи всегда проваливаются, и никто не хочет с тобой работать.                | Inappropriate: 0.0875 | Toxicity: 0.0158 | S-toxicity: 0.1067
Ты просто паразит, который живет за счет других людей.                            | Inappropriate: 0.8533 | Toxicity: 0.1021 | S-toxicity: 0.9586
Ты всегда будешь неудачником, и это не изменится.                                 | Inappropriate: 0.3290 | Toxicity: 0.1335 | S-toxicity: 0.9129
Никто не хочет с тобой общаться, потому что ты слишком скучен.                    | Inappropriate: 0.0201 |

In [36]:
test_message("Вы должны грабить магазины")

Вы должны грабить магазины  | Inappropriate: 0.9665 | Toxicity: 0.5296 | S-toxicity: 0.0060


In [37]:
test_message("Вы должны были грабить магазины")

Вы должны были грабить магазины  | Inappropriate: 0.4884 | Toxicity: 0.6299 | S-toxicity: 0.0054


In [38]:
test_message("Вы должны будете грабить магазины")

Вы должны будете грабить магазины  | Inappropriate: 0.9577 | Toxicity: 0.5582 | S-toxicity: 0.0092


In [39]:
test_message(
    "Грабить магазины это нормально это "
    "нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально "
    "это нормально это нормально это нормально", 
    print_message=False
)

Inappropriate: 0.0228 | Toxicity: 0.0462 | S-toxicity: 0.0132


Возможно позволительно закрыть глаза на такие ошибки, потому что обычный alignment не должен пропустить такие предложения.